### 実装アイディアを出すための実験

In [1]:
import os
# 環境によって変更
os.chdir('/home/jovyan/work/notebook')

In [2]:
class CFG:
    path = "../model"    
    # path = "../../output"
    base_file_path = "../input/sub20241221.csv"
    
    # 焼きなましのパラメータ
    temperature=100
    end_temperature=0.01
    cooling_rate=0.9999
    iteration=int(1e7)
    batch_size=32
    neighbor_types = ["swap", "insert", "inverse", "slide"]

In [3]:
import numpy as np
import pandas as pd
import os
import time
import csv

from tqdm.notebook import tqdm

import random
from typing import List, Callable
import itertools, math
from typing import List, Tuple

from utils import PerplexityCalculator
# 可視化用
import matplotlib.pyplot as plt

### データ読み込み

In [68]:
df = pd.read_csv(CFG.base_file_path)

In [5]:
sub_df = df.copy()
sub_df["best_value"] = 0

In [6]:
df

,id,text
0,0,reindeer mistletoe elf gingerbread family adve...
1,1,reindeer sleep walk the night and drive mistle...
2,2,jingle yuletide carol cheer holly stocking chi...
3,3,sleigh the holly jingle unwrap gifts of magi r...
4,4,peppermint candy milk chocolate eggnog fruitca...
5,5,poinsettia yuletide eggnog fruitcake gingerbre...


In [7]:
# LOAD GEMMA SCORER
scorer = PerplexityCalculator(f'{CFG.path}/gemma_2_9b')

cuda


Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

Some parameters are on the meta device because they were offloaded to the cpu.


In [71]:
# text = "fruitcake and cookie and bake and milk"
# text = "and and and fruitcake cookie bake milk"
text = "the season of peace and joy with peppermint candy milk chocolate eggnog fruitcake cookie wrapping paper wreath bow poinsettia angel toy workshop puzzle game kaggle in that it not you as we dream to wish from night star wonder believe hope have doll candle fireplace hohoho greeting card snowglobe merry"
scorer.get_perplexity([text], batch_size=1)[0]

370.20707083592094

In [51]:
text = df.iloc[5]["text"]

# textを単語に分割
words = text.split()

# 単語の出現回数をカウント
from collections import Counter
word_count = Counter(words)

In [69]:
df.iloc[4]["text"]

'peppermint candy milk chocolate eggnog fruitcake wrapping paper wreath bow poinsettia angel kaggle toy workshop puzzle game the of and to in as that it with not you from we have doll candle fireplace cookie wish dream night star wonder believe hope peace joy season merry hohoho greeting card snowglobe'

In [ ]:
words_length = len(words)

# # word1, word2, text, scoreの列を持つDataFrameを作成
# df = pd.DataFrame(columns=["word1", "word2", "text", "score"])

data = []
for i in tqdm(range(words_length)):
    text_list = []
    word1_list = []
    word2_list = []
    for j in range(words_length):
        if i == j:
            continue
        # words[i]とwords[j]をスペースで結合
        text = words[i] + " " + words[j]
        text_list.append(text)
        word1_list.append(words[i])
        word2_list.append(words[j])
        
    # textのスコアを計算
    score = scorer.get_perplexity(text_list, batch_size=words_length)
    
    for j in range(words_length-1):
        if i == j:
            continue
            
        data.append([word1_list[j], word2_list[j], text_list[j], score[j]])

  0%|          | 0/100 [00:00<?, ?it/s]

In [54]:
tdf = pd.DataFrame(data, columns=["word1", "word2", "text", "score"])

In [55]:
tdf.sort_values("score", inplace=True)

In [56]:
tdf.iloc[:30]

,word1,word2,text,score
197,eggnog,yuletide,eggnog yuletide,837.539771
135,yuletide,scrooge,yuletide scrooge,1322.792088
2745,ornament,yuletide,ornament yuletide,1343.623033
883,ornament,yuletide,ornament yuletide,1343.623033
3823,scrooge,yuletide,scrooge yuletide,1380.869635
98,yuletide,poinsettia,yuletide poinsettia,1510.675982
118,yuletide,mistletoe,yuletide mistletoe,1627.062059
233,eggnog,scrooge,eggnog scrooge,1843.702856
2157,mistletoe,yuletide,mistletoe yuletide,2009.153464
0,poinsettia,eggnog,poinsettia eggnog,2032.836729
